In [7]:
import tflearn
import numpy as np
import argparse
import pickle
import string
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score
import preprocessor as p
from collections import Counter
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix 
from tensorflow.contrib import learn
from tflearn.data_utils import to_categorical, pad_sequences
from scipy import stats
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
import tensorflow as tf
import os
os.environ['KERAS_BACKEND']='theano'
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model,Sequential
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers, optimizers

Using Theano backend.


In [8]:
#These Functions are for loading the data
def load_data(filename):
    print("Loading data from file: " + filename)
    data = pickle.load(open(filename, 'rb'))
    data
    x_text = []
    labels = [] 
    for i in range(len(data)):
        x_text.append(p.tokenize((data[i]['text'])))
        labels.append(data[i]['label'])
    return x_text,labels

def get_filename(dataset):
    global HASH_REMOVE
    if(dataset=="twitter"):
        HASH_REMOVE = True
        EPOCHS = 10
        BATCH_SIZE = 128
        MAX_FEATURES = 2
        filename = "/home/yogesh/Desktop/Minor/data/twitter_data.pkl"
    return filename

In [9]:
data_1 = "twitter"
data_2 = "wiki"
model_type ="blstm_attention"
vector_type = "sswe"
embed_size = 50
oversampling_rate = 3
max_document_length=None
EMBED_SIZE = 50
EPOCHS = 8
BATCH_SIZE = 128
MAX_FEATURES = 2
NUM_CLASSES = 2
DROPOUT = 0.25
LEARN_RATE = 0.01
#HASH_REMOVE=True

In [10]:
x_text, labels = load_data(get_filename(data_1))
dict1 = {'racism':1,'sexism':1,'none':0} #Transfer learning only two classes
labels = [dict1[b] for b in labels]        
racism = [i for i in range(len(labels)) if labels[i]==2]
sexism = [i for i in range(len(labels)) if labels[i]==1]
x_text = x_text + [x_text[x] for x in racism]*(oversampling_rate-1)+ [x_text[x] for x in sexism]*(oversampling_rate-1)
labels = labels + [2 for i in range(len(racism))]*(oversampling_rate-1) + [1 for i in range(len(sexism))]*(oversampling_rate-1)
print("Counter after oversampling")
from collections import Counter
print(Counter(labels))

Loading data from file: /home/yogesh/Desktop/Minor/data/twitter_data.pkl
Counter after oversampling
Counter({1: 15162, 0: 11036})


In [11]:
#splitting the data into training set and testing set
X_train, X_test, Y_train, Y_test = train_test_split( x_text, labels, random_state=121, test_size=0.10)

In [12]:
post_length = np.array([len(x.split(" ")) for x in x_text])
if(data_1 != "twitter"):
    max_document_length = int(np.percentile(post_length, 95))
else:
    max_document_length = max(post_length)
print("Document length : " + str(max_document_length))

Document length : 38


In [13]:
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length, MAX_FEATURES)
vocab_processor = vocab_processor.fit(x_text)
trainX = np.array(list(vocab_processor.transform(X_train)))
testX = np.array(list(vocab_processor.transform(X_test)))

vocab_size = len(vocab_processor.vocabulary_)
print("Vocabulary Size: {:d}".format(vocab_size))

vocab = vocab_processor.vocabulary_._mapping
trainY = np.asarray(Y_train)
testY = np.asarray(Y_test)
       
trainX = pad_sequences(trainX, maxlen=max_document_length, value=0.)
testX = pad_sequences(testX, maxlen=max_document_length, value=0.)
trainY = to_categorical(trainY, nb_classes=NUM_CLASSES)
testY = to_categorical(testY, nb_classes=NUM_CLASSES)

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Vocabulary Size: 8894


In [14]:
print("Running Model: " + model_type + " with word vector initiliazed with " + vector_type + " word vectors.")

#Attention Layer Class for blstm attention model
class AttLayer(Layer):

    def __init__(self, **kwargs):
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='kernel', 
                                      shape=(input_shape[-1],),
                                      initializer='random_normal',
                                      trainable=True)
        super(AttLayer, self).build(input_shape)

    def call(self, x, mask=None):
        eij = K.tanh(K.dot(x, self.W))
        
        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

model = Sequential()
model.add(Embedding(vocab_size, embed_size, input_length=trainX.shape[1]))
model.add(Dropout(0.25))
model.add(Bidirectional(LSTM(embed_size, return_sequences=True)))
model.add(AttLayer())
model.add(Dropout(0.50))
model.add(Dense(NUM_CLASSES, activation='softmax'))
adam = optimizers.Adam(lr=LEARN_RATE, beta_1=0.9, beta_2=0.999)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Running Model: blstm_attention with word vector initiliazed with sswe word vectors.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 38, 50)            444700    
_________________________________________________________________
dropout_1 (Dropout)          (None, 38, 50)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 38, 100)           40400     
_________________________________________________________________
att_layer_1 (AttLayer)       (None, 100)               100       
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 485,402
Trainable params: 485,402
Non-traina

In [9]:
print("Word vectors used: " + vector_type)
def get_embedding_weights(filename, sep):
    embed_dict = {}
    file = open(filename,'r')
    for line in file.readlines():
        row = line.strip().split(sep)
        embed_dict[row[0]] = row[1:]
    print('Loaded from file: ' + str(filename))
    file.close()
    return embed_dict

def map_embedding_weights(embed, vocab, embed_size):
    vocab_size = len(vocab)
    embeddingWeights = np.zeros((vocab_size , embed_size))
    n = 0
    words_missed = []
    for k, v in vocab.items():
        try:
            embeddingWeights[v] = embed[k]
        except:
            n += 1
            words_missed.append(k)
            pass
    print("%d embedding missed"%n, " of " , vocab_size)
    return embeddingWeights

def get_embeddings_dict(vector_type, emb_dim, data):
    if vector_type == 'sswe':
        emb_dim==50
        sep = '\t'
        vector_file = '/home/yogesh/Desktop/Minor/embedding-results/sswe-u.txt'
    else:
        print ("ERROR: Please specify a correst model or SSWE cannot be loaded with embed size of: " + str(emb_dim))
        return None
    
    embed = get_embedding_weights(vector_file, sep)
    return embed
#TRAINING THE MODEL
model.layers[0].set_weights([map_embedding_weights(get_embeddings_dict(vector_type, embed_size, data_1), vocab, embed_size)])
model.fit(trainX, trainY, epochs=EPOCHS, shuffle=True, batch_size=BATCH_SIZE, verbose=1)

Word vectors used: sswe
Loaded from file: /home/yogesh/Desktop/Minor/embedding-results/sswe-u.txt
1159 embedding missed  of  8894
Epoch 1/8
23578/23578 [==============================] - 35s 1ms/step - loss: 0.5828 - acc: 0.6849
Epoch 2/8
23578/23578 [==============================] - 35s 1ms/step - loss: 0.4697 - acc: 0.7757
Epoch 3/8
23578/23578 [==============================] - 36s 2ms/step - loss: 0.3682 - acc: 0.8377
Epoch 4/8
23578/23578 [==============================] - 36s 2ms/step - loss: 0.3015 - acc: 0.8737
Epoch 5/8
23578/23578 [==============================] - 37s 2ms/step - loss: 0.2691 - acc: 0.8887
Epoch 6/8
23578/23578 [==============================] - 37s 2ms/step - loss: 0.2432 - acc: 0.9006
Epoch 7/8
23578/23578 [==============================] - 37s 2ms/step - loss: 0.2185 - acc: 0.9138
Epoch 8/8
23578/23578 [==============================] - 38s 2ms/step - loss: 0.1972 - acc: 0.9224


In [10]:
scores = model.evaluate(testX, testY, verbose=0)

In [11]:
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 91.68%


In [32]:
from keras.models import model_from_json
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [9]:
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json,custom_objects={'AttLayer': AttLayer})
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [10]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
scores = loaded_model.evaluate(testX, testY, verbose=0)

In [12]:
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 91.68%


In [19]:
#code to check model on test data
temp = loaded_model.predict(testX)
y_pred  = np.argmax(temp, 1)
for i in range(len(y_pred)):
    if y_pred[i]==1:
        print("Racism or Sexism!")
    else :
        print("None!")

Racism or Sexism!
None!
None!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
None!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
None!
None!
None!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
None!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
None!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
None!
None!
None!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None

None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
None!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
None!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
None!
None!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Racism or Sexism!
Racism or Sexism!
None!
Racism or Sexism!
Raci

In [17]:
tweet1='pretty sure the number of holes is constant @ashleekelts "this road has more holes in it than a whore has on a saturday night." -kaylies dad'
tokens=p.tokenize(tweet1)
l=[]
l.append(tokens)
arr=np.array(list(vocab_processor.transform(l)))
print(arr)
res=np.argmax(loaded_model.predict(arr),1)
print(res)
if res[0]==1:
    print("Sexist or Racist Comment!")
else :
    print("None!")

[[ 169  211    2  756   11 3040    4 2794    1   29 1501   72   67 3040
    16   15   88    7 1625   72   26    7 2900  402 6169 1188    0    0
     0    0    0    0    0    0    0    0    0    0]]
[1]
Sexist or Racist Comment!


In [43]:
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='kumaryogesh', api_key='8G1LXSapjB8VYt8YG0C2')
flags = ['Racism/Sexism', 'Neutral']
cnt1 = Y_test.count(1)
cnt2 = Y_test.count(0)
values=[cnt1,cnt2]
trace = go.Pie(labels=flags, values=values)
py.iplot([trace], filename='basic_pie_chart')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~kumaryogesh/0 or inside your plot.ly account where it is named 'basic_pie_chart'


In [45]:
import plotly.plotly as py
import plotly.graph_objs as go

fig = {
  "data": [
    {
      "values": [9941,13637],
      "labels": [
        "Neutral",
        "Racism/Sexism"
      ],
      "domain": {"column": 0},
      "name": "Tweets",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    },
    {
      "values": [1095,1525],
      "labels": [
        "Neutral",
        "Racism/Sexism"
      ],
      "text":["Tweets"],
      "textposition":"inside",
      "domain": {"column": 1},
      "name": "Tweets",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    }],
  "layout": {
        "title":"Training and Testing Data",
        "grid": {"rows": 1, "columns": 2},
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Train",
                "x": 0.20,
                "y": 0.5
            },
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Test",
                "x": 0.8,
                "y": 0.5
            }
        ]
    }
}
py.iplot(fig, filename='donut')